In [0]:
# Install Pyomo 
%pip install pyomo --quiet

In [0]:
dbutils.library.restartPython()

In [0]:
import pyomo.environ as pyo
import pandas as pd
import numpy as np
import mlflow
import os

In [0]:
%sh bash install_cbc.sh

In [0]:
A = ['A1', 'A2', 'A3', 'A4', 'A5']
V = ['V1', 'V2', 'V3']
E = [('A1', 'V1'), ('A1', 'V2'), 
     ('A2', 'V1'), ('A2', 'V2'), 
     ('A3', 'V1'), ('A3', 'V2'), ('A3', 'V3'), 
     ('A4', 'V2'), ('A4', 'V3'), 
     ('A5', 'V3')]

# Properties of V
f = {'V1': 0.1, 'V2': 0.15, 'V3': 0.2}  # profit margin
d = {'V1': 200, 'V2': 150, 'V3': 50}    # demand 
s = {'V1': 20, 'V2': 15, 'V3': 5}       # stock

# Properties of A
c = {'A1': 150, 'A2': 100, 'A3': 200, 'A4': 150, 'A5': 50}  # capacity

In [0]:
def build_model(ttr=10):
    
    model = pyo.ConcreteModel()  # Create a concrete optimization model
    model.A = pyo.Set(initialize=A)  
    model.V = pyo.Set(initialize=V)  
    model.E = pyo.Set(within=model.A * model.V, initialize=E)
    
    model.f = pyo.Param(model.V, initialize=f, mutable=True)
    model.d = pyo.Param(model.V, initialize=d, mutable=True)
    model.s = pyo.Param(model.V, initialize=s, mutable=True)
    model.c = pyo.Param(model.A, initialize=c, mutable=True)
    
    model.y_av = pyo.Var(model.E, domain=pyo.NonNegativeIntegers)
    model.l = pyo.Var(model.V, domain=pyo.NonNegativeIntegers)

    def unit_lost(m, v):
        return sum(m.y_av[a, v] for (a,v) in m.E) + m.l[v] >= m.d[v] * ttr - m.s[v]
    model.unit_lost = pyo.Constraint(model.V, rule=unit_lost)

    def total_capacity(m, a):
        return sum(m.y_av[a, v] for (a,v) in m.E) <= m.c[a] * ttr
    model.total_capacity = pyo.Constraint(model.A, rule=total_capacity)
    
    # Objective: total cost
    model.obj = pyo.Objective(
        expr = sum(model.f[v] * model.l[v] for v in model.V),
        sense=pyo.minimize
    )

    return model  # Return the constructed model

In [0]:
model = build_model()
solver = pyo.SolverFactory("cbc")
result = solver.solve(model)

In [0]:
total_loss = pyo.value(model.obj)
total_loss